In [ ]:
# C:/Users/bento/AppData/Local/ESRI/conda/envs/arcgispro-py3-clone/python.exe 

In [3]:
# import libraries
import pandas as pd
import numpy as np
import os
import arcpy
import re 
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile


# Data from https://feederwatch.org/explore/raw-dataset-requests/

def get_species_codes() -> pd.DataFrame:
    """
    This function queries the Species Codes sheet from the FeederWatch Data Dictionary. The
    data is available through an excel sheet provided in the data website. This data will be 
    used to access the corresponding names and families of the different species codes.
    Returns a pandas dataframe of species (Fields: species_code, species_name, family)
    """
    # First, set up the url for the data dictionary (Google Drive).
    # Credit goes to the following StackOverflow answer for re-formatting the url:
    # https://stackoverflow.com/questions/56611698/pandas-how-to-read-csv-file-from-google-drive-public
    url = 'https://drive.google.com/file/d/1kHmx2XhA2MJtEyTNMpwqTQEnoa9M7Il2/view?usp=sharing'
    url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
    # Read the Excel Sheet with the Species Codes
    species = pd.read_excel(url, sheet_name='Species Codes', header=1)
    # Filter and rename columns
    species = species[['SPECIES_CODE', 'PRI_COM_NAME_INDXD', 'FAMILY']]/
        .rename(columns={'SPECIES_CODE':'species_code', 
                         'PRI_COM_NAME_INDXD':'species_name',
                         'FAMILY':'family'})
    return species

def clean_fw_data(data:pd.DataFrame, 
                  birds:pd.DataFrame, 
                  sub_national_code:list=[]) -> pd.DataFrame:
    """
    This function cleans the FeederWatch data so that it only contains 
    relevent fields, accurate (valid) data, specified birds, and specified locations.
    Args:
    - data: a pandas dataframe; raw data downloaded from FeederWatch site
    - birds: species data - output of get_species_codes(). 
             * Note: It can be a subset of this data (e.g., a specific family)
    - sub_national_code: list of `subnational1_code` fields to filter to 
    Returns a subset of the original data input, with cleaned field names.
    """
    # All available names of fields in dataset
    all_names = ['loc_id', 'latitude', 'longitude', 'subnational1_code', 
                 'entry_technique', 'sub_id', 'obs_id', 'month', 'day',
                 'year', 'proj_period_id', 'species_code', 'how_many',
                 'valid', 'reviewed', 'plus_code', 'day1_am', 'day1_pm',
                 'day2_am', 'day2_pm', 'effort_hrs_atleast', 
                 'snow_dep_atleast', 'data_entry_method']
    # Output names of fields in dataset (to be kept)
    out_names = ['species_code', 'species_name', 'how_many', 'latitude', 
                 'longitude', 'subnational1_code', 'date']
    # Preprocessing (fix column names, include/exclude fields)
    data.rename(columns=str.lower, inplace=True)
    other_names = [n for n in all_names if n not in data.columns]
    data = data.assign(**{name:np.nan for name in other_names if len(other_names) > 0})
    # Filter Data by valid, no plus_code, species, optional location
    data = data.query(f'valid == 1 & plus_code != 1 & species_code == @birds.species_code.to_list()')
    if sub_national_code is not None:
        data = data.loc[data.subnational1_code.isin(sub_national_code)]
    # Join with species (to get species name)
    data = pd.merge(data, birds, how='left', on='species_code')
    # Date formatting
    data['date'] = pd.to_datetime(dict(year=data.year, 
                                       month=data.month, 
                                       day=data.day))
    # Return, Ensuring correct order, specific output columns, sorted
    return data[out_names].sort_values(by=['date', 'species_name'], ascending=[True, True])

def getFeedWatcherData(outfile:str,
                       tfs:list, 
                       birds:pd.DataFrame, 
                       sub_national_code:list=[], 
                       out_dir:str='data', 
                       file_suffix:str='',
                       save_:bool=True,
                       min_year:int=2017,
                       max_year:int=2019) -> pd.DataFrame:
    """
    Gets FeederWatch data from website. When reading directly from the URLs 
    and saving the output, this can take a while (depending on internet speed).
    Each independent query (by date range) is saved to a gzipped .csv file,
    so if the process is interrupted or re-run, it can be read directly from
    that file instead of re-downloaded. Data is also cleaned/filtered (using 
    `clean_fw_data()`), then concatenated and saved to a final .csv file.
    Args: 
    - outfile: Final output file name
    - tfs: Time-frames to get data for
    - birds: Species data (Optionally) pre-filtered (e.g., by family)
    - sub_national_code: (Optionally) filter by subnational1_code (e.g., U.S. State)
    - out_dir: Directory in which to save data
    - file_suffix: Suffix of file names
    - save_: Whether or not to save the output to a gzipped file
    - min_year: minimum year to filter data
    - max_year: maximum year to filter data
    Returns a pandas dataframe of the selected FeederWatch bird data
    """
    final_out_file = os.path.join(out_dir, outfile)
    # First check if the file already exists
    if os.path.isfile(final_out_file):
        out = pd.read_csv(final_out_file)
        out['date'] = pd.to_datetime(out.date)
    else:
        df_lis = list()
        for i in np.arange(0, len(tfs)):
            # Read Data (either from URL, or from previously saved data if available)
            tf = tfs[i]
            out_file = os.path.join(out_dir, f'FW_{tf}_{file_suffix}.csv.gz')
            if not os.path.isfile(out_file):
                url = 'https://clo-pfw-prod.s3.us-west-2.amazonaws.com/data/PFW_' + tf + '_public.csv'
                print(f"Getting {tf} data from {url}")
                # Read/Clean data
                data = clean_fw_data(data=pd.read_csv(url), 
                                    birds=birds, 
                                    sub_national_code=sub_national_code)
                if save_:
                    # If not previously cached, save as gzip
                    print(f"Saving {tf} data to {out_file}")
                    data.to_csv(out_file, compression='gzip', index=False)
            else:
                print(f"Reading {tf} data from {out_file}")
                data = pd.read_csv(out_file, compression='gzip')
                data['date'] = pd.to_datetime(data.date)
            # Append to list
            df_lis.append(data)
        # Combine list into single dataframe
        print("Concatenating list of dataframes")
        out = pd.concat(df_lis)
        # Filter by date
        out = out[out['date'].dt.year >= min_year]
        out = out[out['date'].dt.year <= max_year]
        # Save to file
        out.to_csv(final_out_file, index=False)
    return out

# Timeframes available in FeederWatch:
# '1988_1995', '1996_2000', '2001_2005', '2006_2010', 
# '2011_2015', '2016_2020', '2021'

# Select 2017 - 2019 (Covered by 2019 Land Cover Raster)
DATA_TIMEFRAMES = ['2016_2020']
# All Species
SPECIES = get_species_codes()
# Woodpecker Family
WOODPECKERS = SPECIES.loc[SPECIES['family'] == 'Picidae (Woodpeckers)']

fw = getFeedWatcherData(outfile="FW_woodpeckers_NC.csv",
                        tfs=DATA_TIMEFRAMES,
                        birds=WOODPECKERS,
                        sub_national_code=['US-NC'],
                        out_dir='data',
                        file_suffix='woodpeckers_NC',
                        save_=True)


In [4]:
class Species():
    def __init__(self, dataframe:pd.DataFrame) -> None:
        self.species_code = dataframe.species_code.to_list()
        self.species_name = dataframe.species_name.to_list()
        self.family = dataframe.family.to_list()

class Bird(Species):
    def __init__(self, dataframe:pd.DataFrame, bird_name:str) -> None:
        super().__init__(dataframe)
        # Get index from original dataframe
        bird_idx = np.array([bird_name == b for b in self.species_name])
        # Create attributes
        self.code = str(np.array(self.species_code)[bird_idx][0])
        self.name = str(np.array(self.species_name)[bird_idx][0])
        self.family = str(np.array(self.family)[bird_idx][0])
        # Adjust name for formatted feature class name attribute
        name_parts = self.name.split(', ')
        self.formatted_name = re.sub("[()]", "", name_parts[1] + "_" + name_parts[0])\
                .replace(" ", "_").replace("-", "_")
        self.fc_name = f"FW_{self.formatted_name}_NC"


In [5]:
# Define Globals; Setup
PROJ_PATH = "C:/Users/bento/OneDrive/code_and_data/ncsu-mgist/courses/gis_540/final_project/woodpecker-nc"
DB_PATH = "fw_GDB.gdb"
# Create File Geodatabase
if not os.path.exists(os.path.join(PROJ_PATH, DB_PATH)):
    arcpy.CreateFileGDB_management(PROJ_PATH, DB_PATH)
arcpy.env.workspace = os.path.join(PROJ_PATH, DB_PATH)
DATA_PATH = os.path.join(PROJ_PATH, "data")
FW_FILE = "FW_woodpeckers_NC.csv"
BASE_FC = "FW_woodpeckers_NC"
EXISTING_FCS = arcpy.ListFeatureClasses()
# Projected Coordinate System
COORD_SYSTEM = arcpy.SpatialReference("NAD 1983 StatePlane North Carolina FIPS 3200 (US Feet)")

def batchBirdAnalysis(fw_file:str, 
                      base_fc:str,
                      existing_fcs:list,
                      out_coordinate_system:arcpy.SpatialReference, 
                      data_path:str,
                      fw_df:pd.DataFrame,
                      species_df:pd.DataFrame) -> None:
    """
    Batch processing and analysis of FeederWatch bird data. 
    Steps:
    1) Creates Feature Class from .csv file in file Geodatabase
    2) Adds projection, saving to new Feature Class
    3) Filters Projected Feature Class by species, saving individual
       species to their own Feature Classes in the database
    Args: 
    - fw_file: FeederWatch data .csv file
    - base_fc: Base Feature Class name
    - existing_fcs: List of existing Feature Classes already saved to the 
      database (if they already exist, they will be skipped during batch
      processing)
    - out_coordinate_system: Projected coordinate system
    - data_path: Path to feederwatch data
    - fw_df: FeederWatch dataframe
    - species_df: Species dataframe
    """
    # Create base feature class
    if base_fc not in existing_fcs:
        arcpy.management.XYTableToPoint(in_table=os.path.join(data_path, fw_file),
                                        out_feature_class=base_fc,
                                        x_field="longitude", 
                                        y_field="latitude")
    # Add projection to base FC
    if f"{base_fc}_projected" not in existing_fcs:
        arcpy.management.Project(base_fc, 
                                f"{base_fc}_projected", 
                                out_coordinate_system)
    # Add to GDB by species
    for species_name in fw_df.species_name.unique():
        brd = Bird(dataframe=species_df, bird_name=species_name)
        if brd.fc_name not in existing_fcs:
            print(f'Adding {brd.name} to gdb...')
            arcpy.analysis.Select(f"{base_fc}_projected", 
                                brd.fc_name, 
                                f"species_name = '{brd.name}'")
        
        
batchBirdAnalysis(fw_file=FW_FILE, 
                  base_fc=BASE_FC,
                  existing_fcs=EXISTING_FCS, 
                  out_coordinate_system=COORD_SYSTEM,
                  data_path=DATA_PATH,
                  fw_df=fw,
                  species_df=WOODPECKERS)


In [37]:
import urllib.request
import zipfile

# Get Raster Data
# Credit to https://svaderia.github.io/articles/downloading-and-unzipping-a-zipfile/

# https://www.lib.ncsu.edu/gis/nlcd
if not os.path.isdir('data/NC_Land_Cover/'):
    # 2019 only 
    zipurl = 'https://gisdata.lib.ncsu.edu/fedgov/mrlc/nlcd2019/NC_NLCD2019only.zip'
    # 2001 - 2019, every 3 years (NOT IN USE)
    # "https://drive.google.com/uc?id=1555Ox4664hH0kFlakGQwi1nzxrMcC61o&confirm=t&uuid=0edbf032-c3ba-45fe-b111-c3752b7cf8ae&at=ALgDtsw-mvJqXBLq4JMNZJ-5g2b7:1676943369421"
    with urlopen(zipurl) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall('data/NC_Land_Cover')

if 'nc_nlcd2019_Resample_1k' not in arcpy.ListRasters():
    # Resample cell size of raster(s), save to gdb
    arcpy.management.Resample("data/NC_Land_Cover/nc_nlcd2019", 
                            "nc_nlcd2019_Resample_1k", 
                            "1000 1000", 
                            "MAJORITY")

# Connect to NCSU network (on-campus or through VPN)
if "nc250" not in os.listdir("data/DEM"):
    # URL to the North Carolina boundary 250m DEM
    url = "https://gisdata.lib.ncsu.edu/DEM/nc250.zip"
    zip_file_name = "nc250.zip"
    # Download the zip file from the URL
    urllib.request.urlretrieve(url, zip_file_name)
    # Extract the contents of the zip file to a directory named "data/DEM"
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall("data/DEM")
        
if "nc250" not in arcpy.ListRasters():
    arcpy.management.CopyRaster("data/DEM/nc250", "nc250", nodata_value="-3.402823e+38")

In [53]:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/presence-only-prediction.htm

arcpy.stats.PresenceOnlyPrediction(input_point_features="FW_Downy_Woodpecker_NC", 
                                   contains_background="PRESENCE_ONLY_POINTS", 
                                   explanatory_variables=None, #TODO
                                   presence_indicator_field=None, 
                                   distance_features=None, 
                                   explanatory_rasters=[["nc_nlcd2019_Resample_1k", "true"], 
                                                        ["nc250", "false"]], 
                                   basis_expansion_functions="HINGE", 
                                   number_knots=10, 
                                   study_area_type="RASTER_EXTENT", 
                                   study_area_polygon=None, 
                                   spatial_thinning="THINNING", 
                                   thinning_distance_band="2500 Meters", 
                                   number_of_iterations=20, 
                                   relative_weight=100,  #1-100
                                   link_function="CLOGLOG", 
                                   presence_probability_cutoff=0.5, 
                                   output_trained_features="Downy_Woodpecker_NC_Trained_Features", 
                                   output_trained_raster="Downy_Woodpecker_NC_Trained_Raster", 
                                   output_response_curve_table="Downy_Woodpecker_NC_Response_Curve", 
                                   output_sensitivity_table="Downy_Woodpecker_NC_Sensitivity_Table", 
                                   features_to_predict=None, 
                                   output_pred_features=None, 
                                   output_pred_raster=None, 
                                   explanatory_variable_matching=None, 
                                   explanatory_distance_matching=None, 
                                   explanatory_rasters_matching=None, 
                                   allow_predictions_outside_of_data_ranges="ALLOWED", 
                                   resampling_scheme="RANDOM", 
                                   number_of_groups=5)

<Result 'C:/Users/bento/OneDrive/code_and_data/ncsu-mgist/courses/gis_540/final_project/woodpecker-nc\\fw_GDB.gdb\\Downy_Woodpecker_NC_Trained_Features'>

In [52]:

for fc in ["Downy_Woodpecker_NC_Trained_Features", 
           "Downy_Woodpecker_NC_Trained_Raster", 
           "Downy_Woodpecker_NC_Response_Curve", 
           "Downy_Woodpecker_NC_Sensitivity_Table"]:
    if fc in arcpy.ListTables() or fc in arcpy.ListFeatureClasses():
        print(fc)
        arcpy.Delete_management(fc)
        


In [3]:

vars = ["ppt", "tmax", "tmin"]
yrs = [2017, 2018, 2019]

pairs = list()
pairs = [(v, y) for v in vars for y in yrs if (v, y) not in pairs]
pairs

[('ppt', 2017),
 ('ppt', 2018),
 ('ppt', 2019),
 ('tmax', 2017),
 ('tmax', 2018),
 ('tmax', 2019),
 ('tmin', 2017),
 ('tmin', 2018),
 ('tmin', 2019)]